# Import required modules

In [19]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn import cross_validation
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns
import glob
import os
from sklearn.cross_validation import cross_val_score
plt.style.use('ggplot')

# Define Functions

In [20]:
#A function to calculate the results
def result(x):
	conditions = [
	(x['GoalsFor'] > x['GoalsAgainst']),
	(x['GoalsFor'] == x['GoalsAgainst']),
	(x['GoalsFor'] < x['GoalsAgainst'])
	]
	choices = [1,0,-1]
	x['Result'] = np.select(conditions, choices)
	return x

In [21]:
def joined_result(x):
    if x == 'H':
        return 1
    elif x == 'D':
        return 0
    elif x == 'A':
        return -1

# Data Preperation

Read CSV Data

In [22]:
full_stats=pd.read_csv(r'C:\Users\dbrown\Documents\Misc\Football\16_17\overall.csv')
for year in ['0001','0102','0203','0304','0405','0506','0607','0708','0809','0910','1011','1112','1213','1314','1415','1516','1617','1718']:
    for div in ['E0','E1','E2','E3','SC0','SC1','SC2','SC3','B1','F1','F2','SP1','SP2','I1','I2','D1','D2','N1','P1','G1','T1']:
        url = 'http://www.football-data.co.uk/mmz4281/1718/%s.csv' % div
        #results = results.append(pd.read_csv(url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']))
        full_stats = full_stats.append(pd.read_csv(url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']))
    
fixtures_url="http://www.football-data.co.uk/fixtures.csv"
fixtures=pd.read_csv(fixtures_url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

Define base dataframes

In [23]:
#full_stats = full_results[['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']]
#fixtures = fixtures[['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']]
full_results_combo = full_stats.append(fixtures)
full_results_combo.drop_duplicates(subset=['Date','Div','HomeTeam','AwayTeam'],keep='last',inplace=True)
home_stats = full_results_combo.set_index(['Date','Div','HomeTeam'])
away_stats = full_results_combo.set_index(['Date','Div','AwayTeam'])

Work on dataframes

In [24]:
home_stats = home_stats[['FTHG','FTAG']]
home_stats.rename(columns={'FTHG':'GoalsFor','FTAG':'GoalsAgainst'},inplace='True')
home_stats['GoalDiff'] = home_stats['GoalsFor'] - home_stats['GoalsAgainst']
away_stats = away_stats[['FTAG','FTHG']]
away_stats.rename(columns={'FTAG':'GoalsFor','FTHG':'GoalsAgainst'},inplace='True')
away_stats['GoalDiff'] = away_stats['GoalsFor'] - away_stats['GoalsAgainst']
#Set dataframes with the result
home_stats = result(home_stats)
away_stats = result(away_stats)
#Rename columns in Home and Away stats
home_stats.rename(columns={'GoalsFor':'HomeGoalsFor','GoalsAgainst':'HomeGoalsAgainst','GoalDiff':'HomeGoalDiff','Result':'HomeResult'},inplace='True')
away_stats.rename(columns={'GoalsFor':'AwayGoalsFor','GoalsAgainst':'AwayGoalsAgainst','GoalDiff':'AwayGoalDiff','Result':'AwayResult'},inplace='True')

Set 2-6 game form values

In [25]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        home_stats[HomeGameForm] = home_stats['HomeResult'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        home_stats[HomeGameGoalsFor] = home_stats['HomeGoalsFor'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        home_stats[HomeGameGoalDiff] = home_stats['HomeGoalDiff'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameForm] = away_stats['AwayResult'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameGoalsFor] = away_stats['AwayGoalsFor'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        away_stats[AwayGameGoalDiff] = away_stats['AwayGoalDiff'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))

Merge Home and Away Stats

In [26]:
home_noindex = home_stats.reset_index()
away_noindex = away_stats.reset_index()
home_merge = pd.merge(full_results_combo, home_noindex, on=['Date','Div','HomeTeam'])
home_merge.rename(columns={'HomeTeam_x':'HomeTeam','AwayTeam_x':'AwayTeam'},inplace=True)
away_merge = pd.merge(full_results_combo, away_noindex, on=['Date','Div','AwayTeam'])
away_merge.rename(columns={'HomeTeam_x':'HomeTeam','AwayTeam_x':'AwayTeam'},inplace=True)
home_away_merge = pd.merge(home_merge, away_merge, on=['Date','Div','AwayTeam'])
home_away_merge.rename(columns={'HomeTeam_x':'HomeTeam'},inplace=True)
home_away_merge = home_away_merge[['Date', 'Div','HomeTeam','AwayTeam','HomeGoalsFor', 'HomeGoalsAgainst', 'HomeGoalDiff', 'HomeResult','Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff','Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'AwayGoalsFor', 'AwayGoalsAgainst',
       'AwayGoalDiff', 'AwayResult', 'Away2GameForm', 'Away2GameGoalsFor',
       'Away2GameGoalDiff', 'Away3GameForm', 'Away3GameGoalsFor',
       'Away3GameGoalDiff', 'Away4GameForm', 'Away4GameGoalsFor',
       'Away4GameGoalDiff', 'Away5GameForm', 'Away5GameGoalsFor',
       'Away5GameGoalDiff', 'Away6GameForm', 'Away6GameGoalsFor',
       'Away6GameGoalDiff']]

Set relevant columns and clean data

In [27]:
home_away_merge_idx = home_away_merge.set_index(['Date','Div','HomeTeam','AwayTeam'])
#home_away_merge_idx_relevant = home_away_merge_idx[['HomeGoalsFor','AwayGoalsFor','HomeResult','3GameFormDiff']]
#home_away_merge_cleaned = home_away_merge_idx_relevant.dropna()

Set 2-6 game form difference values

In [28]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        GameFormDiff = "%sGameFormDiff" % (num)
        GameGoalsForDiff = "%sGameGoalsForDiff" % (num)
        GameGoalDiffDiff = "%sGameGoalDiffDiff" % (num)
        home_away_merge_idx[GameFormDiff] = home_away_merge_idx[HomeGameForm] - home_away_merge_idx[AwayGameForm]
        home_away_merge_idx[GameGoalsForDiff] = home_away_merge_idx[HomeGameGoalsFor] - home_away_merge_idx[AwayGameGoalsFor]
        home_away_merge_idx[GameGoalDiffDiff] = home_away_merge_idx[HomeGameGoalDiff] - home_away_merge_idx[AwayGameGoalDiff]

Deal with NaN values

In [29]:
home_away_merge_idx_zeroed = home_away_merge_idx.fillna(0)
home_away_merge_idx_dropped = home_away_merge_idx.dropna()
home_away_merge_idx_zeroed_reset = home_away_merge_idx_zeroed.reset_index()

Get the latest stats

In [30]:
home_stats_latest = home_away_merge_idx_zeroed_reset.groupby('HomeTeam').last()
home_stats_latest.reset_index(inplace=True)
home_stats_latest.set_index(['Date','Div','HomeTeam','AwayTeam'],inplace=True)
latest_fixtures = home_stats_latest[home_stats_latest["HomeResult"]==0]

In [44]:
home_away_merge_idx_dropped.columns

Index(['HomeGoalsFor', 'HomeGoalsAgainst', 'HomeGoalDiff', 'HomeResult',
       'Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff',
       'Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'AwayGoalsFor', 'AwayGoalsAgainst', 'AwayGoalDiff', 'AwayResult',
       'Away2GameForm', 'Away2GameGoalsFor', 'Away2GameGoalDiff',
       'Away3GameForm', 'Away3GameGoalsFor', 'Away3GameGoalDiff',
       'Away4GameForm', 'Away4GameGoalsFor', 'Away4GameGoalDiff',
       'Away5GameForm', 'Away5GameGoalsFor', 'Away5GameGoalDiff',
       'Away6GameForm', 'Away6GameGoalsFor', 'Away6GameGoalDiff',
       '2GameFormDiff', '2GameGoalsForDiff', '2GameGoalDiffDiff',
       '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff',
       '4GameFormDiff', '4GameGoalsForDiff', '4GameGoalDiffDif

# Logistic Regression on HomeResult

In [45]:
y = home_away_merge_idx_dropped['HomeResult']
X = home_away_merge_idx_dropped[[
    '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff']]
logreg = LogisticRegression()

In [46]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.8)
logreg.fit(X_train, y_train)
logreg.score(X_train, y_train)
cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean()

0.47610877373676336

In [52]:
y = home_away_merge_idx_dropped['HomeResult']
X = home_away_merge_idx_dropped[[
    'Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff',
       'Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'Away2GameForm', 'Away2GameGoalsFor', 'Away2GameGoalDiff',
       'Away3GameForm', 'Away3GameGoalsFor', 'Away3GameGoalDiff',
       'Away4GameForm', 'Away4GameGoalsFor', 'Away4GameGoalDiff',
       'Away5GameForm', 'Away5GameGoalsFor', 'Away5GameGoalDiff',
       'Away6GameForm', 'Away6GameGoalsFor', 'Away6GameGoalDiff',
       '2GameFormDiff', '2GameGoalsForDiff', '2GameGoalDiffDiff',
       '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff',
       '4GameFormDiff', '4GameGoalsForDiff', '4GameGoalDiffDiff',
       '5GameFormDiff', '5GameGoalsForDiff', '5GameGoalDiffDiff',
       '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff']]

logreg = LogisticRegression()

#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.9)
logreg.fit(X, y)
logreg.score(X, y)

y_pred = logreg.predict(X)
#metrics.accuracy_score(y_test, y_pred)

#cross_val_score(logreg, X, y, cv=10, scoring='accuracy')

#cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean()

#y_pred_prob = logreg.predict_proba(X_test)[:, -1]

#logreg.predict_proba(X_test)[:, -3]

#y_pred_prob

latest_fixtures_cut = latest_fixtures[[
    'Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff',
       'Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'Away2GameForm', 'Away2GameGoalsFor', 'Away2GameGoalDiff',
       'Away3GameForm', 'Away3GameGoalsFor', 'Away3GameGoalDiff',
       'Away4GameForm', 'Away4GameGoalsFor', 'Away4GameGoalDiff',
       'Away5GameForm', 'Away5GameGoalsFor', 'Away5GameGoalDiff',
       'Away6GameForm', 'Away6GameGoalsFor', 'Away6GameGoalDiff',
       '2GameFormDiff', '2GameGoalsForDiff', '2GameGoalDiffDiff',
       '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff',
       '4GameFormDiff', '4GameGoalsForDiff', '4GameGoalDiffDiff',
       '5GameFormDiff', '5GameGoalsForDiff', '5GameGoalDiffDiff',
       '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff']]
probs = logreg.predict_proba(latest_fixtures_cut)
probs_df = pd.DataFrame(probs)
probs_df.rename(columns={0:'AwayWinProb',1:'DrawProb',2:'HomeWinProb'},inplace='True')
latest_fixtures_cut_noind = latest_fixtures_cut.reset_index()
pred_join = latest_fixtures_cut_noind.join(probs_df)
pred_join_cut = pred_join[["Date","Div","HomeTeam","AwayTeam","HomeWinProb"]]

In [53]:
pred_join_cut[pred_join_cut["HomeWinProb"]>0.65]

,Date,Div,HomeTeam,AwayTeam,HomeWinProb
50,24/02/18,SP2,Cadiz,Lorca,0.682370
97,23/02/18,T1,Galatasaray,Bursaspor,0.677629
117,24/02/18,I1,Inter,Benevento,0.671614
157,16/02/18,F2,Nimes,Tours,0.684622
181,17/02/18,SC3,Peterhead,Berwick,0.672224
195,24/02/18,SP1,Real Madrid,Alaves,0.686613
221,24/02/18,SC1,St Mirren,Brechin,0.688234
235,20/02/18,F1,Troyes,Dijon,0.665265
